**Problem 1 [10] Pick-and-Place Robot:** Consider using reinforcement learning to control the motion of a robot arm in a repetitive pick-and-place task. If we want to learn movements that are fast and smooth, the learning agent will have to control the motors directly and obtain feedback about the current positions and velocities of the mechanical linkages. Design the reinforcement learning problem as an MDP, define states, actions, rewards with reasoning.

## Problem 1: Pick-and-Place Robot as an MDP

### Introduction

Modeling pick-and-place robot problem as a Markov Decision Process (MDP).

An MDP is defined as:

(S, A, P, R)

Where:
- S = States  
- A = Actions  
- P = Transition probability  
- R = Reward function  

---

### 1. States (S)

The state should describe the current condition of the robot.

As per the question the agent receives feedback about positions and velocities, we define the state as:
s_t = (q_t, v_t, g_t)

Where:
- q_t = joint positions  
- v_t = joint velocities  
- g_t = gripper status (open or closed)

Reason:
The robot’s next movement depends only on its current position, velocity, and gripper status.  
This satisfies the Markov property (future depends only on present state and action).

---

### 2. Actions (A)

The agent controls the motors directly.

So the actions are motor torque commands:

a_t = (tau_1, tau_2, ..., tau_n)

Where:
- tau_i = torque applied to joint i

Reason:
Motor torque control allows smooth and precise movements.  
This is necessary for learning smooth robot motion.

---

### 3. Transition Probability (P)

The transition function is:

P(s' | s, a)

This means:
Given the current state and action, what is the probability of moving to the next state?

In this robot system, transitions depend on:
- Robot mechanics (physics)
- Motor commands
- Small noise in sensors or motors

Because of noise, the system can be slightly stochastic.

Also:

The sum of all transition probabilities must equal 1.

---

### 4. Reward Function (R)

The question says we want movements that are fast and smooth.

So the reward should:

- Give a large positive reward when the object is successfully placed.
- Penalize large velocities (to encourage smooth motion).
- Penalize large torques (to avoid jerky movement).
- Add a small time penalty (to encourage faster completion).

Example reward:

R = +Goal_Reward 
    - alpha * (velocity^2) 
    - beta * (torque^2) 
    - time_penalty

Reason:
This reward structure helps the robot learn:
- Fast movement  
- Smooth motion  
- Energy-efficient control  
- Successful task completion  

---

### Conclusion

The pick-and-place robot can be modeled as an MDP where:

- States represent joint positions, velocities, and gripper status.
- Actions are motor torque commands.
- Transitions follow robot dynamics.
- Rewards encourage fast, smooth, and successful movement

## Problem 2: 2x2 Gridworld - Value Iteration (2 Iterations)

**Environment Details:**
2x2 Gridworld – Value Iteration

State Space: s1, s2, s3, s4  
Action Space: up, down, left, right  

Rewards:
R(s1)=5
R(s2)=10
R(s3)=1
R(s4)=2

Discount Factor: γ = 0.9

Value Iteration Formula:

V_{k+1}(s) = max_a [ R(s) + γ V_k(s') ]

Transitions are deterministic.
If action hits wall → agent stays in same state.
Initial values: V0(s) = 0

**Grid Layout:**    
s1   s2     
s3   s4     

**Coordinates:**        
(0,0) = s1      
(0,1) = s2      
(1,0) = s3      
(1,1) = s4      

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import random
import matplotlib.patches as patches
import copy
import os



In [ ]:
class GridWorldValueIteration:
    
    def __init__(self, gamma=0.9):
        self.gamma = gamma
        
        # Grid size
        self.rows = 2
        self.cols = 2
        
        # Rewards
        self.rewards = np.array([[5, 10],
                                 [1,  2]])
        
        # Initialize Value Function V0 = 0
        self.V = np.zeros((self.rows, self.cols))
        
        # Actions: (row_change, col_change)
        self.actions = {
            "up": (-1, 0),
            "down": (1, 0),
            "left": (0, -1),
            "right": (0, 1)
        }
    
    def is_valid(self, r, c):
        return 0 <= r < self.rows and 0 <= c < self.cols
    
    def get_next_state(self, r, c, action):
        dr, dc = self.actions[action]
        new_r, new_c = r + dr, c + dc
        
        if self.is_valid(new_r, new_c):
            return new_r, new_c
        else:
            return r, c  # stay if wall
    
    def iterate(self, iteration_number):
        new_V = np.zeros_like(self.V)
        
        print(f"\n========== Iteration {iteration_number} ==========")
        
        for r in range(self.rows):
            for c in range(self.cols):
                
                action_values = []
                
                print(f"\nState ({r},{c})  Reward = {self.rewards[r,c]}")
                
                for action in self.actions:
                    next_r, next_c = self.get_next_state(r, c, action)
                    
                    value = self.rewards[r,c] + \
                            self.gamma * self.V[next_r, next_c]
                    
                    action_values.append(value)
                    
                    print(f" Action: {action} -> Next: ({next_r},{next_c}) "
                          f" | Value = {self.rewards[r,c]} + "
                          f"{self.gamma}*{self.V[next_r,next_c]:.2f} "
                          f"= {value:.2f}")
                
                new_V[r,c] = max(action_values)
                
                print(f" Updated V({r},{c}) = {new_V[r,c]:.2f}")
        
        self.V = new_V
        print("\nUpdated Value Function:")
        print(self.V)
    
    def run(self, num_iterations=2):
        print("Initial Value Function V0:")
        print(self.V)
        
        for i in range(1, num_iterations+1):
            self.iterate(i)

# Run
grid = GridWorldValueIteration(gamma=0.9)
grid.run(num_iterations=2)

Since initial V0 = 0 for all states:        

V1(s) = R(s) + γ * 0                

Therefore:

V1(s1) = 5      
V1(s2) = 10     
V1(s3) = 1      
V1(s4) = 2      

Now we use V1 values.

Example for s1: 

Possible next states:   
Up -> s1    
Left -> s1  
Right -> s2 
Down -> s3  

Compute:    

5 + 0.9 * 5 = 9.5     
5 + 0.9 * 10 = 14     
5 + 0.9 * 1 = 5.9     

Take maximum:   

V2(s1) = 14 

Same process is repeated for all states.    

Final V2:   

[[14, 19],  
 [5.5, 11]] 

Iteration 1:        
[[ 5. 10.]          
 [ 1.  2.]]     

Iteration 2:        
[[14. 19.]      
 [ 5.5 11.]]        

## Problem 3 – 5x5 Gridworld using Value Iteration

In this problem, we model a 5x5 Gridworld as a Markov Decision Process (MDP).

### Environment Setup:
# 5x5 Gridworld – Value Iteration   
**State Types:**        
- Goal State (Terminal) = +10 reward        
- Grey States = -5 reward       
- Regular States = -1 reward        

**Grey States:**        
(0,4)       
(1,2)       
(3,0)       

**Goal State:**     
(4,4)       

**Discount Factor:** gamma = 0.9        

**Bellman Optimality Equation:**        

V(s) = max_a [ R(s) + gamma * V(s') ]       

If action hits wall → stay in same state.       

In [ ]:
## Problem 3: 5x5 Gridworld MDP

class GridWorld5x5:

    def __init__(self, gamma=0.9, log_folder="log"):

        self.rows = 5
        self.cols = 5
        self.gamma = gamma

        self.goal_state = (4, 4)
        self.grey_states = [(0, 4), (1, 2), (3, 0)]

        self.rewards = self.create_reward_matrix()

        self.actions = {
            "up": (-1, 0),
            "down": (1, 0),
            "left": (0, -1),
            "right": (0, 1)
        }

        # -------- LOG FOLDER SETUP --------
        self.log_folder = log_folder
        os.makedirs(self.log_folder, exist_ok=True)

    # ---------------------------------------------------
    # Create reward matrix
    # ---------------------------------------------------
    def create_reward_matrix(self):
        R = np.full((self.rows, self.cols), -1)

        for s in self.grey_states:
            R[s] = -5

        R[self.goal_state] = 10
        return R

    # ---------------------------------------------------
    def is_valid(self, r, c):
        return 0 <= r < self.rows and 0 <= c < self.cols

    # ---------------------------------------------------
    def get_next_state(self, r, c, action):
        dr, dc = self.actions[action]
        nr, nc = r + dr, c + dc

        if self.is_valid(nr, nc):
            return nr, nc
        return r, c

    # ===================================================
    # STANDARD VALUE ITERATION WITH LOGGING
    # ===================================================
    def value_iteration_standard(self, theta=0.001, log_file="problem3_iteration_log.txt"):

        V = np.zeros((self.rows, self.cols))
        iterations = 0

        start_time = time.time()

        full_log_path = os.path.join(self.log_folder, log_file)
        with open(full_log_path, "w") as f:
            f.write("===== STANDARD VALUE ITERATION =====\n")

            while True:
                delta = 0
                new_V = np.copy(V)
                iterations += 1

                f.write(f"\nIteration {iterations}\n")

                for r in range(self.rows):
                    for c in range(self.cols):

                        if (r, c) == self.goal_state:
                            new_V[r, c] = self.rewards[r, c]
                            continue

                        action_values = []

                        for action in self.actions:
                            nr, nc = self.get_next_state(r, c, action)
                            value = self.rewards[r, c] + \
                                    self.gamma * V[nr, nc]
                            action_values.append(value)

                        new_V[r, c] = max(action_values)
                        delta = max(delta, abs(new_V[r, c] - V[r, c]))

                V = new_V

                # Log matrix
                for row in V:
                    f.write(" ".join(f"{v:7.2f}" for v in row) + "\n")

                if delta < theta:
                    f.write("\nConverged.\n")
                    break

        end_time = time.time()
        return V, iterations, end_time - start_time

    # ===================================================
    # IN-PLACE VALUE ITERATION
    # ===================================================
    def value_iteration_inplace(self, theta=0.001):

        V = np.zeros((self.rows, self.cols))
        iterations = 0
        start_time = time.time()

        while True:
            delta = 0
            iterations += 1

            for r in range(self.rows):
                for c in range(self.cols):

                    if (r, c) == self.goal_state:
                        V[r, c] = self.rewards[r, c]
                        continue

                    action_values = []

                    for action in self.actions:
                        nr, nc = self.get_next_state(r, c, action)
                        value = self.rewards[r, c] + \
                                self.gamma * V[nr, nc]
                        action_values.append(value)

                    new_value = max(action_values)
                    delta = max(delta, abs(new_value - V[r, c]))
                    V[r, c] = new_value

            if delta < theta:
                break

        end_time = time.time()
        return V, iterations, end_time - start_time

    # ---------------------------------------------------
    # Extract policy
    # ---------------------------------------------------
    def extract_policy(self, V):
        policy = np.empty((self.rows, self.cols), dtype=object)

        for r in range(self.rows):
            for c in range(self.cols):

                if (r, c) == self.goal_state:
                    policy[r, c] = "G"
                    continue

                best_action = None
                best_value = -float("inf")

                for action in self.actions:
                    nr, nc = self.get_next_state(r, c, action)
                    value = self.rewards[r, c] + \
                            self.gamma * V[nr, nc]

                    if value > best_value:
                        best_value = value
                        best_action = action

                policy[r, c] = best_action

        return policy

    # ---------------------------------------------------
    # Log Policy Matrix + Agent Movement
    # ---------------------------------------------------
    def log_policy_and_agent(self, policy, V,
                             log_file="problem3_agent_log.txt",
                             start_state=(0, 0)):

        full_log_path = os.path.join(self.log_folder, log_file)
        with open(full_log_path, "a") as f:

            f.write("\n\n===== FINAL OPTIMAL VALUE FUNCTION =====\n")
            for row in V:
                f.write(" ".join(f"{v:7.2f}" for v in row) + "\n")

            f.write("\n===== OPTIMAL POLICY MATRIX =====\n")
            for r in range(self.rows):
                row_policy = []
                for c in range(self.cols):
                    if (r, c) in self.grey_states:
                        row_policy.append("X")
                    elif (r, c) == self.goal_state:
                        row_policy.append("G")
                    else:
                        row_policy.append(policy[r, c][0].upper())
                f.write(" ".join(row_policy) + "\n")

            # Agent movement
            f.write("\n===== AGENT MOVEMENT =====\n")

            current_state = start_state
            step = 0

            while current_state != self.goal_state:
                step += 1
                r, c = current_state

                grid_display = [["." for _ in range(self.cols)]
                                for _ in range(self.rows)]

                for g in self.grey_states:
                    grid_display[g[0]][g[1]] = "X"

                grid_display[self.goal_state[0]][self.goal_state[1]] = "G"
                grid_display[r][c] = "A"

                f.write(f"\nStep {step}\n")
                for row in grid_display:
                    f.write(" ".join(row) + "\n")

                action = policy[r, c]
                current_state = self.get_next_state(r, c, action)

            f.write("\nGoal Reached.\n")


# ---------------------------------------------------
# Grid Plot Function
# ---------------------------------------------------
def draw_grid(grid_obj, title, values=None, policy=None):

    fig, ax = plt.subplots()
    ax.set_title(title)

    for r in range(grid_obj.rows):
        for c in range(grid_obj.cols):

            color = "white"

            if (r, c) in grid_obj.grey_states:
                color = "darkgrey"
            elif (r, c) == grid_obj.goal_state:
                color = "lightgreen"

            rect = patches.Rectangle((c, grid_obj.rows - 1 - r), 1, 1,
                                     edgecolor='black',
                                     facecolor=color)
            ax.add_patch(rect)

            if values is not None:
                ax.text(c + 0.5, grid_obj.rows - 1 - r + 0.5,
                        round(values[r, c], 2),
                        ha='center', va='center')

            if policy is not None:
                if (r, c) not in grid_obj.grey_states and \
                   (r, c) != grid_obj.goal_state:

                    arrow_map = {
                        "up": (0, 0.3),
                        "down": (0, -0.3),
                        "left": (-0.3, 0),
                        "right": (0.3, 0)
                    }
                    dx, dy = arrow_map[policy[r, c]]
                    ax.arrow(c + 0.5, grid_obj.rows - 1 - r + 0.5,
                             dx, dy,
                             head_width=0.15,
                             length_includes_head=True)

    ax.set_xlim(0, grid_obj.cols)
    ax.set_ylim(0, grid_obj.rows)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect('equal')

    plt.show()


# ===================================================
# RUN EVERYTHING
# ===================================================

grid = GridWorld5x5()

# Standard VI (with logging)
V_standard, iter_std, time_std = grid.value_iteration_standard()

# In-place VI
V_inplace, iter_in, time_in = grid.value_iteration_inplace()

# Extract policy
pi_standard = grid.extract_policy(V_standard)

# Log policy + agent movement into same file
grid.log_policy_and_agent(pi_standard, V_standard)

# Display figures
draw_grid(grid, "Initial Grid")
draw_grid(grid, "Optimal State Values V*", values=V_standard)
draw_grid(grid, "Optimal Policy π*", policy=pi_standard)

print("\nStandard Iterations:", iter_std)
print("In-place Iterations:", iter_in)
print("Standard Time:", round(time_std, 6))
print("In-place Time:", round(time_in, 6))
print("Same V*?", np.allclose(V_standard, V_inplace))

5 by 5 Gridworld using Value Iteration.

Normal states give −1 reward, grey states give −5, and the goal gives +10.  
The agent can move in four directions and stays in place if it hits a wall.     

Strted with all values equal to zero and repeatedly updated them using the Bellman equation until they stopped changing.     

After convergence, I selected the best action in each state to get the optimal policy.      

The program shows four grids: the initial grid, reward grid, optimal values, and optimal policy.        

Finally, I simulated the agent, and it follows the arrows to reach the goal     

## Problem 4 [35] Problem Statement Off-policy Monte Carlo with Importance Sampling: We will use the same environment, states, actions, and rewards in Problem 3.

In [ ]:
import numpy as np
import random
import time


class GridWorld5x5MonteCarlo:
    """
    5x5 Gridworld for Off-Policy Monte Carlo with Importance Sampling

    State Types:
        Regular state = -1 reward
        Grey state    = -5 reward
        Goal state    = +10 reward

    Behavior policy: random
    Target policy: greedy
    """

    def __init__(self, discount_factor=0.9, log_folder="log"):

        self.rows = 5
        self.cols = 5
        self.gamma = discount_factor

        self.goal_state = (4, 4)
        self.grey_states = [(0, 4), (1, 2), (3, 0)]

        self.action_space = ["up", "down", "left", "right"]

        self.reward_matrix = self._create_reward_matrix()

        # ---------- LOG FOLDER ----------
        self.log_folder = log_folder
        os.makedirs(self.log_folder, exist_ok=True)

    # ---------------------------------------------------
    # Create reward matrix
    # ---------------------------------------------------
    def _create_reward_matrix(self):

        rewards = np.full((self.rows, self.cols), -1)

        for state in self.grey_states:
            rewards[state] = -5

        rewards[self.goal_state] = 10

        return rewards

    # ---------------------------------------------------
    # Check if state is valid
    # ---------------------------------------------------
    def _is_valid_state(self, row, col):
        return 0 <= row < self.rows and 0 <= col < self.cols

    # ---------------------------------------------------
    # Deterministic transition function
    # ---------------------------------------------------
    def _get_next_state(self, current_state, action):

        row, col = current_state

        action_moves = {
            "up": (-1, 0),
            "down": (1, 0),
            "left": (0, -1),
            "right": (0, 1)
        }

        delta_row, delta_col = action_moves[action]
        next_row = row + delta_row
        next_col = col + delta_col

        if self._is_valid_state(next_row, next_col):
            return (next_row, next_col)

        return current_state

    # ---------------------------------------------------
    # Behavior Policy: Uniform Random
    # ---------------------------------------------------
    def behavior_policy(self, state):
        return random.choice(self.action_space)

    # ---------------------------------------------------
    # Generate one episode using behavior policy
    # ---------------------------------------------------
    def generate_episode(self, max_steps=100,
                         path_log_file="problem4_agent_path_log.txt"):

        episode = []
        current_state = (0, 0)

        full_path = os.path.join(self.log_folder, path_log_file)

        with open(full_path, "a") as file:

            file.write("\n===== New Episode =====\n")

            for step in range(max_steps):

                action = self.behavior_policy(current_state)
                next_state = self._get_next_state(current_state, action)

                reward = self.reward_matrix[next_state]

                episode.append((current_state, action, reward))

                # Log agent path
                file.write(f"Step {step+1}: "
                           f"State={current_state}, "
                           f"Action={action}, "
                           f"Reward={reward}, "
                           f"Next={next_state}\n")

                if next_state == self.goal_state:
                    file.write("Goal Reached.\n")
                    break

                current_state = next_state

        return episode

    # ===================================================
    # Off-Policy Monte Carlo with Weighted Importance Sampling
    # ===================================================
    full_value_path = os.path.join(self.log_folder, value_log_file)
    with open(full_value_path, "w") as file:

        state_value_function = np.zeros((self.rows, self.cols))
        cumulative_weights = np.zeros((self.rows, self.cols))

        # Initialize target policy randomly
        target_policy = {}
        for row in range(self.rows):
            for col in range(self.cols):
                target_policy[(row, col)] = random.choice(self.action_space)

        start_time = time.time()

        with open(value_log_file, "w") as file:

            for episode_index in range(number_of_episodes):

                episode = self.generate_episode()

                return_G = 0
                importance_weight = 1

                # Process episode backward
                for time_step in reversed(range(len(episode))):

                    state, action, reward = episode[time_step]
                    row, col = state

                    return_G = self.gamma * return_G + reward

                    cumulative_weights[row, col] += importance_weight

                    # Incremental update rule
                    state_value_function[row, col] += (
                        importance_weight / cumulative_weights[row, col]
                    ) * (return_G - state_value_function[row, col])

                    # Improve target policy greedily
                    best_action = None
                    best_value = -float("inf")

                    for possible_action in self.action_space:
                        next_state = self._get_next_state(state,
                                                          possible_action)
                        next_row, next_col = next_state

                        estimated_value = (
                            self.reward_matrix[next_state] +
                            self.gamma *
                            state_value_function[next_row, next_col]
                        )

                        if estimated_value > best_value:
                            best_value = estimated_value
                            best_action = possible_action

                    target_policy[state] = best_action

                    # Stop if behavior deviates from target
                    if action != target_policy[state]:
                        break

                    # Update importance sampling ratio
                    importance_weight *= 1 / (1/4)

                # Log value matrix every 500 episodes
                if (episode_index + 1) % 500 == 0:
                    file.write(f"\nEpisode {episode_index+1}\n")
                    for row_values in state_value_function:
                        file.write(" ".join(
                            f"{value:7.2f}" for value in row_values
                        ) + "\n")

        end_time = time.time()

        return state_value_function, target_policy, \
               end_time - start_time


def draw_grid_problem4(grid_object, title,
                    value_function=None,
                    policy=None):

    fig, ax = plt.subplots()
    ax.set_title(title)

    for row in range(grid_object.rows):
        for col in range(grid_object.cols):

            color = "white"

            if (row, col) in grid_object.grey_states:
                color = "darkgrey"
            elif (row, col) == grid_object.goal_state:
                color = "lightgreen"

            rectangle = patches.Rectangle(
                (col, grid_object.rows - 1 - row),
                1, 1,
                edgecolor='black',
                facecolor=color
            )

            ax.add_patch(rectangle)

            # Display value function
            if value_function is not None:
                ax.text(col + 0.5,
                        grid_object.rows - 1 - row + 0.5,
                        round(value_function[row, col], 2),
                        ha='center', va='center')

            # Display policy arrows
            if policy is not None:

                if (row, col) not in grid_object.grey_states and \
                (row, col) != grid_object.goal_state:

                    arrow_map = {
                        "up": (0, 0.3),
                        "down": (0, -0.3),
                        "left": (-0.3, 0),
                        "right": (0.3, 0)
                    }

                    dx, dy = arrow_map[policy[(row, col)]]

                    ax.arrow(col + 0.5,
                            grid_object.rows - 1 - row + 0.5,
                            dx, dy,
                            head_width=0.15,
                            length_includes_head=True)

    ax.set_xlim(0, grid_object.cols)
    ax.set_ylim(0, grid_object.rows)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect('equal')

    plt.show()

grid_mc = GridWorld5x5MonteCarlo(
    discount_factor=0.9,
    log_folder="log"
)

estimated_value_function, estimated_policy, mc_time = \
    grid_mc.off_policy_monte_carlo(number_of_episodes=5000)

print("\n===== MONTE CARLO ESTIMATED VALUE FUNCTION =====")
print(np.round(estimated_value_function, 2))
print("Optimization Time:", round(mc_time, 4), "seconds")

# 1Initial Grid
draw_grid_problem4(grid_mc, "Problem 4 - Initial Grid")

# Monte Carlo Estimated Value Function
draw_grid_problem4(grid_mc,
                   "Problem 4 - Monte Carlo Estimated V(s)",
                   value_function=estimated_value_function)

# Monte Carlo Estimated Policy
draw_grid_problem4(grid_mc,
                   "Problem 4 - Monte Carlo Estimated Policy",
                   policy=estimated_policy)

## Comparison With Value Iteration

| Aspect      | Value Iteration       | Monte Carlo                  |
| ----------- | --------------------- | ---------------------------- |
| Uses model? | Yes                   | No                           |
| Speed       | Faster                | Slower                       |
| Convergence | Deterministic         | High variance                |
| Iterations  | ~10–20                | Thousands                    |
| Complexity  | O(S × A × iterations) | O(episodes × episode_length) |
